In [1]:
# 分布类别不检测


In [1]:
import argparse
import os
import numpy as np
from keras.layers import Conv2D, Input, BatchNormalization, LeakyReLU, ZeroPadding2D, UpSampling2D
from keras.layers.merge import add, concatenate
from keras.models import Model
import struct
import xml.dom.minidom

from keras.layers import Reshape, Activation, Conv2D, Input, MaxPooling2D, BatchNormalization, Flatten, Dense, Lambda
from keras.layers import LeakyReLU,AvgPool2D
from keras.callbacks import EarlyStopping, ModelCheckpoint#, TensorBoard
from keras.optimizers import SGD, Adam, RMSprop
from keras.layers.merge import concatenate
import matplotlib.pyplot as plt
import keras.backend as K
config = K.tf.ConfigProto()
config.gpu_options.allow_growth = True
session = K.tf.Session(config=config)
K.set_session(session)

import tensorflow as tf
import imgaug as ia
from tqdm import tqdm
from imgaug import augmenters as iaa
import numpy as np
import pickle
import os, cv2
from preprocessing import parse_annotation, BatchGenerator
from utils import WeightReader, decode_netout2, draw_boxes

Using TensorFlow backend.


In [2]:
ANCHORS  = [[0.36,0.17,  0.67,0.24, 0.18,0.105], [0.65,0.16, 0.197,0.06, 0.39,0.076],[0.18,0.02, 0.35,0.026, 0.56,0.03]]

LABELS = ['tableRegion', 'figureRegion', 'formulaRegion']
GRID_H,  GRID_W  = 15 , 15
IMAGE_H, IMAGE_W = GRID_H*64,  GRID_W*64


ANCHORS = np.asarray(ANCHORS,dtype=np.float32)*np.asarray([[GRID_H], [GRID_H*2], [GRID_H*4]])
BOX              = 3
CLASS            = len(LABELS)
CLASS_WEIGHTS    = np.ones(CLASS, dtype='float32')
OBJ_THRESHOLD    = 0.6#0.5
NMS_THRESHOLD    = 0.3#0.45

NO_OBJECT_SCALE  = 1.0
OBJECT_SCALE     = 5.0
COORD_SCALE      = 1.0
CLASS_SCALE      = 1.0

BATCH_SIZE       = 8

In [3]:
weights_path = 'yolov3.weights'                      
train_image_folder = '/home/zhuleike/data/science/Train/JPEGImages/'
train_annot_folder = '/home/zhuleike/data/science/Train/Annotations/'
test_image_folder = '/home/zhuleike/data/science/Test/JPEGImages/'
test_annot_folder = '/home/zhuleike/data/science/Test/annotations/'
#valid_annot_folder = '/home/trunk/RTrunk0/zhuleike/data/VOCdevkit/VOC2012/Annotations/valid/'

In [4]:


class BoundBox:
    def __init__(self, xmin, ymin, xmax, ymax, objness = None, classes = None):
        self.xmin = xmin
        self.ymin = ymin
        self.xmax = xmax
        self.ymax = ymax
        
        self.objness = objness
        self.classes = classes

        self.label = -1
        self.score = -1

    def get_label(self):
        if self.label == -1:
            self.label = np.argmax(self.classes)
        
        return self.label
    
    def get_score(self):
        if self.score == -1:
            self.score = self.classes[self.get_label()]
            
        return self.score

def _conv_block(inp, convs, skip=True):
    x = inp
    count = 0
    i = 0
    for conv in convs:
        if count == (len(convs) - 2) and skip:
            skip_connection = x
        count += 1
        

        x = Conv2D(conv['filter'], 
                   kernel_size = conv['kernel'], 
                   strides=conv['stride'], 
                   padding='same', 
                   name='conv_' + str(conv['layer_idx']), 
                   use_bias=False if conv['bnorm'] else True)(x)
        if conv['bnorm']: x = BatchNormalization(epsilon=0.001, name='bnorm_' + str(conv['layer_idx']))(x)
        if conv['leaky']: x = LeakyReLU(alpha=0.1, name='leaky_' + str(conv['layer_idx']))(x)

    return add([skip_connection, x]) if skip else x

def _interval_overlap(interval_a, interval_b):
    x1, x2 = interval_a
    x3, x4 = interval_b

    if x3 < x1:
        if x4 < x1:
            return 0
        else:
            return min(x2,x4) - x1
    else:
        if x2 < x3:
             return 0
        else:
            return min(x2,x4) - x3          

def _sigmoid(x):
    return 1. / (1. + np.exp(-x))

def bbox_iou(box1, box2):
    intersect_w = _interval_overlap([box1.xmin, box1.xmax], [box2.xmin, box2.xmax])
    intersect_h = _interval_overlap([box1.ymin, box1.ymax], [box2.ymin, box2.ymax])
    
    intersect = intersect_w * intersect_h

    w1, h1 = box1.xmax-box1.xmin, box1.ymax-box1.ymin
    w2, h2 = box2.xmax-box2.xmin, box2.ymax-box2.ymin
    
    union = w1*h1 + w2*h2 - intersect
    
    return float(intersect) / union

def make_yolov3_model():
    input_image = Input(shape=(None, None, 3))

    # Layer  0 => 4
    x = _conv_block(input_image, [{'filter': 32, 'kernel': 3, 'stride': 2, 'bnorm': True, 'leaky': True, 'layer_idx': 0},
                                  {'filter': 64, 'kernel': 3, 'stride': 2, 'bnorm': True, 'leaky': True, 'layer_idx': 1},
                                  {'filter': 32, 'kernel': 1, 'stride': 1, 'bnorm': True, 'leaky': True, 'layer_idx': 2},
                                  {'filter': 64, 'kernel': 3, 'stride': 1, 'bnorm': True, 'leaky': True, 'layer_idx': 3}])

    # Layer  5 => 8
    x = _conv_block(x, [{'filter': 128, 'kernel': 3, 'stride': 2, 'bnorm': True, 'leaky': True, 'layer_idx': 5},
                        {'filter':  64, 'kernel': 1, 'stride': 1, 'bnorm': True, 'leaky': True, 'layer_idx': 6},
                        {'filter': 128, 'kernel': 3, 'stride': 1, 'bnorm': True, 'leaky': True, 'layer_idx': 7}])

    # Layer  9 => 11
    x = _conv_block(x, [{'filter':  64, 'kernel': 1, 'stride': 1, 'bnorm': True, 'leaky': True, 'layer_idx': 9},
                        {'filter': 128, 'kernel': 3, 'stride': 1, 'bnorm': True, 'leaky': True, 'layer_idx': 10}])

    # Layer 12 => 15
    x = _conv_block(x, [{'filter': 256, 'kernel': 3, 'stride': 2, 'bnorm': True, 'leaky': True, 'layer_idx': 12},
                        {'filter': 128, 'kernel': 1, 'stride': 1, 'bnorm': True, 'leaky': True, 'layer_idx': 13},
                        {'filter': 256, 'kernel': 3, 'stride': 1, 'bnorm': True, 'leaky': True, 'layer_idx': 14}])

    # Layer 16 => 36
    
    for i in range(7):
        if i % 2 == 0:
            tempkernel = (5,1)
        else:
            tempkernel = (1,5)
        x = _conv_block(x, [{'filter': 128, 'kernel': 1, 'stride': 1, 'bnorm': True, 'leaky': True, 'layer_idx': 16+i*3},
                            {'filter': 256, 'kernel': tempkernel, 'stride': 1, 'bnorm': True, 'leaky': True, 'layer_idx': 17+i*3}])
        
    skip_36 = x
        
    # Layer 37 => 40
    x = _conv_block(x, [{'filter': 512, 'kernel': 3, 'stride': 2, 'bnorm': True, 'leaky': True, 'layer_idx': 37},
                        {'filter': 256, 'kernel': 1, 'stride': 1, 'bnorm': True, 'leaky': True, 'layer_idx': 38},
                        {'filter': 512, 'kernel': 3, 'stride': 1, 'bnorm': True, 'leaky': True, 'layer_idx': 39}])

    # Layer 41 => 61
    for i in range(7):
        if i % 2 == 0:
            tempkernel = (5,1)
        else:
            tempkernel = (1,5)
        x = _conv_block(x, [{'filter': 256, 'kernel': 1, 'stride': 1, 'bnorm': True, 'leaky': True, 'layer_idx': 41+i*3},
                            {'filter': 512, 'kernel': tempkernel, 'stride': 1, 'bnorm': True, 'leaky': True, 'layer_idx': 42+i*3}])
        
    skip_61 = x
        
    # Layer 62 => 65
    x = _conv_block(x, [{'filter': 1024, 'kernel': 3, 'stride': 2, 'bnorm': True, 'leaky': True, 'layer_idx': 62},
                        {'filter':  512, 'kernel': 1, 'stride': 1, 'bnorm': True, 'leaky': True, 'layer_idx': 63},
                        {'filter': 1024, 'kernel': (1,5), 'stride': 1, 'bnorm': True, 'leaky': True, 'layer_idx': 64}])

    # Layer 66 => 74
    for i in range(3):
        if i % 2 == 0:
            tempkernel = (5,1)
        else:
            tempkernel = (1,5)
        x = _conv_block(x, [{'filter':  512, 'kernel': 1, 'stride': 1, 'bnorm': True, 'leaky': True, 'layer_idx': 66+i*3},
                            {'filter': 1024, 'kernel': tempkernel, 'stride': 1, 'bnorm': True, 'leaky': True, 'layer_idx': 67+i*3}])
        
    # Layer 75 => 79
    x = _conv_block(x, [{'filter':  512, 'kernel': 1, 'stride': 1, 'bnorm': True, 'leaky': True, 'layer_idx': 75},
                        {'filter': 1024, 'kernel': (1,5), 'stride': 1, 'bnorm': True, 'leaky': True, 'layer_idx': 76},
                        {'filter':  512, 'kernel': 1, 'stride': 1, 'bnorm': True, 'leaky': True, 'layer_idx': 77},
                        {'filter': 1024, 'kernel': (5,1), 'stride': 1, 'bnorm': True, 'leaky': True, 'layer_idx': 78},
                        {'filter':  512, 'kernel': 1, 'stride': 1, 'bnorm': True, 'leaky': True, 'layer_idx': 79}], skip=False)

    # Layer 80 => 82
    yolo_82 = _conv_block(x, [{'filter': 1024, 'kernel': 3, 'stride': 1, 'bnorm': True,  'leaky': True,  'layer_idx': 80},
                              {'filter':  3*(5 + CLASS), 'kernel': 1, 'stride': 1, 'bnorm': False, 'leaky': False, 'layer_idx': 81}], skip=False)

    # Layer 83 => 86
    x = _conv_block(x, [{'filter': 256, 'kernel': 1, 'stride': 1, 'bnorm': True, 'leaky': True, 'layer_idx': 84}], skip=False)
    x = UpSampling2D(2)(x)
    x = concatenate([x, skip_61])

    # Layer 87 => 91
    x = _conv_block(x, [{'filter': 256, 'kernel': 1, 'stride': 1, 'bnorm': True, 'leaky': True, 'layer_idx': 87},
                        {'filter': 512, 'kernel': (1,5), 'stride': 1, 'bnorm': True, 'leaky': True, 'layer_idx': 88},
                        {'filter': 256, 'kernel': 1, 'stride': 1, 'bnorm': True, 'leaky': True, 'layer_idx': 89},
                        {'filter': 512, 'kernel': (5,1), 'stride': 1, 'bnorm': True, 'leaky': True, 'layer_idx': 90},
                        {'filter': 256, 'kernel': 1, 'stride': 1, 'bnorm': True, 'leaky': True, 'layer_idx': 91}], skip=False)

    # Layer 92 => 94
    yolo_94 = _conv_block(x, [{'filter': 512, 'kernel': 3, 'stride': 1, 'bnorm': True,  'leaky': True,  'layer_idx': 92},
                              {'filter': 3*(5 + CLASS), 'kernel': 1, 'stride': 1, 'bnorm': False, 'leaky': False, 'layer_idx': 93}], skip=False)

    # Layer 95 => 98
    x = _conv_block(x, [{'filter': 128, 'kernel': 1, 'stride': 1, 'bnorm': True, 'leaky': True,   'layer_idx': 96}], skip=False)
    x = UpSampling2D(2)(x)
    x = concatenate([x, skip_36])

    # Layer 99 => 106
    yolo_106 = _conv_block(x, [{'filter': 128, 'kernel': 1, 'stride': 1, 'bnorm': True,  'leaky': True,  'layer_idx': 99},
                               {'filter': 256, 'kernel': (1,5), 'stride': 1, 'bnorm': True,  'leaky': True,  'layer_idx': 100},
                               {'filter': 128, 'kernel': 1, 'stride': 1, 'bnorm': True,  'leaky': True,  'layer_idx': 101},
                               {'filter': 256, 'kernel': (5,1), 'stride': 1, 'bnorm': True,  'leaky': True,  'layer_idx': 102},
                               {'filter': 128, 'kernel': 1, 'stride': 1, 'bnorm': True,  'leaky': True,  'layer_idx': 103},
                               {'filter': 256, 'kernel': 3, 'stride': 1, 'bnorm': True,  'leaky': True,  'layer_idx': 104},
                               {'filter': 3*(5 + CLASS), 'kernel': 1, 'stride': 1, 'bnorm': False, 'leaky': False, 'layer_idx': 105}], skip=False)
    yolo_82 =  Reshape((GRID_H, GRID_W, BOX, 4 + 1 + CLASS))(yolo_82)
    yolo_94 =  Reshape((GRID_H*2, GRID_W*2, BOX, 4 + 1 + CLASS))(yolo_94)
    yolo_106 =  Reshape((GRID_H*4, GRID_W*4, BOX, 4 + 1 + CLASS))(yolo_106)

    model = Model(input_image, [yolo_82, yolo_94, yolo_106])    
    return model



In [5]:
model = make_yolov3_model()
#model.load_weights('current_best.h5')

In [6]:
model.load_weights('yolov3_18.h5')

In [25]:
#model.load_weights('current_best2.h5')

In [7]:
#model.load_weights('weights_coco_yolov3.h5')


In [7]:
generator_config = {
    'IMAGE_H'         : IMAGE_H, 
    'IMAGE_W'         : IMAGE_W,
    'GRID_H'          : GRID_H,  
    'GRID_W'          : GRID_W,
    'BOX'             : BOX,
    'LABELS'          : LABELS,
    'CLASS'           : len(LABELS),
    'ANCHORS'         : ANCHORS,
    'BATCH_SIZE'      : BATCH_SIZE,
    'TRUE_BOX_BUFFER' : 50,
}

In [8]:
def normalize(image):
    return image / 255.

In [9]:
#from preprocessing import parse_annotation


In [10]:
#train_imgs = parse_annotation(train_annot_folder, train_image_folder, labels=LABELS)
#valid_imgs, seen_train_labels = parse_annotation(valid_annot_folder, train_image_folder, labels=LABELS)
#valid_imgs = train_imgs[::2]

#train_batch = BatchGenerator(train_imgs, generator_config, norm=normalize)
#valid_batch = BatchGenerator(valid_imgs, generator_config, norm=normalize, jitter=False)

In [11]:
#test_imgs = parse_annotation(test_annot_folder, test_image_folder, labels=LABELS)

In [12]:
#model.load_weights('best_pod22.h5')
#all_imgs = parse_annotation(train_annot_folder, train_image_folder, labels=LABELS)

In [13]:
import random

In [14]:
def build_model():
    # 构建卷积神经网络结构
    input_image = Input(shape = (32, 960,3))
    x = Conv2D(32, (3, 3), strides=(1,1), use_bias =False)(input_image)
    x = BatchNormalization()(x)
    x = LeakyReLU(alpha=0.1)(x)
    x = MaxPooling2D(pool_size=(2, 4))(x)
    x = Conv2D(64, (3, 3), strides=(1,1), use_bias =False)(x)
    x = BatchNormalization()(x)
    x = LeakyReLU(alpha=0.1)(x)
    x = MaxPooling2D(pool_size=(2, 4))(x)
    for i in range(0,3):
        channel = min(512, 128*(2**i))
        x = Conv2D(channel*2, (1, 1), use_bias =False, padding='same')(x)
        x = BatchNormalization()(x)
        x = LeakyReLU(alpha=0.1)(x)        
        x = Conv2D(channel, (2, 4), strides=(1,1), use_bias =False, padding='valid')(x) 
        x = BatchNormalization()(x)
        x = MaxPooling2D(pool_size=(1, 2))(x)
        x = LeakyReLU(alpha=0.1)(x)

        
    x = Conv2D(channel, (2, 4), strides=(1,1), use_bias =False, name = 'end1')(x)
    x = BatchNormalization()(x)
    x = LeakyReLU(alpha=0.1)(x)
    x = Conv2D(128, (2, 1), strides=(1,1), use_bias =False, name = 'end2')(x)
    x = BatchNormalization()(x)
    x = LeakyReLU(alpha=0.1)(x)    
    x = Conv2D(2, (1, 1), use_bias =False,activation='sigmoid', name = 'endl3')(x)

    model = Model(inputs=input_image, outputs=x)
    return model


In [15]:
def judge_image_col(image):
    h,w,_ = image.shape
    image2 = image[h//4: h//4*3]
    a = np.sum(np.max(image2)-image2, 0)
    a = np.max(a,-1)
    lena = len(a)
    down = int(450.0/1100*lena)
    uppe = int(600.0/1100*lena)

    middle = np.where(a[down:uppe] < np.mean(a)*0.1)[0]

    if len(middle) == 0:
        return 0
    else:
        return int(np.mean(middle)+down)

In [19]:
filename = 'utils.py'
exec(compile(open(filename, "rb").read(), filename, 'exec'))

In [16]:
formula_model = build_model()
formula_model.load_weights('/home/zhuleike/workspace/bishe/muticlassfy/rotatnew.hdf5')

In [23]:
import time

OBJ_THRESHOLD    = 0.2
NMS_THRESHOLD    = 0.4
start_time = time.time()
doc = xml.dom.minidom.Document() 
#在内存中创建一个空的文档

root = doc.createElement('root') 
def writeimage(imagename):
    singleimage = doc.createElement('document') 
    singleimage.setAttribute('filename', imagename) 
    return singleimage  
# generate the probs and coords for boxes
def write2xml(boxname,  prob, coord):


    nodeManager = doc.createElement(boxname)
    nodeManager.setAttribute('prob', prob)
    nodeName = doc.createElement('Coords')
    nodeName.setAttribute('points', coord)

    nodeManager.appendChild(nodeName)
    return nodeManager

for idx, img_name in enumerate(sorted(os.listdir(test_image_folder))):
    if not img_name.lower().endswith(('.bmp', '.jpeg', '.jpg', '.png', '.tif', '.tiff')):
        continue
    singleimage = writeimage(img_name)
    imagepath = os.path.join(test_image_folder,img_name)

    #plt.figure(figsize=(10,10))
    image = cv2.imread(imagepath)
    col_index = judge_image_col(image)
    if col_index!= 0:
        image_l = image[:,:col_index]/250.0
        image_r = image[:,col_index:]/250.0
    h,w,_ = image.shape
    
    
    input_image = cv2.resize(image, (IMAGE_H, IMAGE_W))
    input_image = input_image / 255.
    input_image = input_image[:,:,::-1]

    
    input_image = np.expand_dims(input_image, 0)

    netout = model.predict(input_image)
    boxes = decode_netout2(netout, 
                          obj_threshold=OBJ_THRESHOLD,
                          nms_threshold=NMS_THRESHOLD,
                          anchors=ANCHORS, 
                          nb_class=CLASS)    
    for box in boxes:
        x1, y1, x2, y2,new_probs  = box.xmin, box.ymin, box.xmax, box.ymax, box.c
        
        key = LABELS[box.get_label()]
        x1, y1, x2, y2 = int(round(x1*w)), int(round(y1*h)), int(round(x2*w)), int(round(y2*h))        
        if x1 < 10 or y1 < 10 or x2> w-10 or y2 > h - 10:
            continue         
         
        if key == LABELS[-1] and new_probs < 0.90:
            if col_index == 0:
                tempimage = image/250.0
                xx1 = x1
                xx2 = x2
            elif x2<= col_index:
                tempimage = np.copy(image_l)
                xx1 = x1
                xx2 = x2               
                
            elif x1 >= col_index:
                tempimage = np.copy(image_r)
                xx1 = x1 - col_index
                xx2 = x2 - col_index
                
            temp = tempimage[y1-2:y2+2]
            temp[:,xx1:xx2,-1] = 0
            temp = cv2.resize(temp, (960, 32))
            temp = np.clip(temp, 0,1)
            temp2 = np.expand_dims(temp, 0)
            temp2 = formula_model.predict(temp2)
            if temp2[0,0,0,1] > 0.9:
                continue
                        
            
        coord = str(x1)+','+str(y1)+' '+str(x2)+','+str(y1)+' '+ str(x1)+','+str(y2)+' '+ str(x2)+','+str(y2)
        
        nodeManager = write2xml( key, str(new_probs), coord)
        singleimage.appendChild(nodeManager)
 
    root.appendChild(singleimage)

doc.appendChild(root)
fp = open('Manager.xml', 'w')
doc.writexml(fp, indent='\t', addindent='\t', newl='\n', encoding="utf-8")
print(time.time() - start_time )

172.29244375228882


In [24]:
fp.close()

In [36]:
x2
#np.where(np.mean(tempimage[middley,:col_index[1]],axis=-1) < 0.1)[0]

978

In [20]:
OBJ_THRESHOLD    = 0.2
NMS_THRESHOLD    = 0.4
import xml.dom.minidom
doc = xml.dom.minidom.Document() 
#在内存中创建一个空的文档
root = doc.createElement('root') 
def writeimage(imagename):
    singleimage = doc.createElement('document') 
    singleimage.setAttribute('filename', imagename) 
    return singleimage  
# generate the probs and coords for boxes
def write2xml(boxname,  prob, coord):


    nodeManager = doc.createElement(boxname)
    nodeManager.setAttribute('prob', prob)
    nodeName = doc.createElement('Coords')
    nodeName.setAttribute('points', coord)

    nodeManager.appendChild(nodeName)
    return nodeManager

for idx, img_name in enumerate(sorted(os.listdir(test_image_folder))):
    if not img_name.lower().endswith(('.bmp', '.jpeg', '.jpg', '.png', '.tif', '.tiff')):
        continue
    singleimage = writeimage(img_name)
    imagepath = os.path.join(test_image_folder,img_name)

    #plt.figure(figsize=(10,10))
    image = cv2.imread(imagepath)
    h,w,_ = image.shape
    
    input_image = cv2.resize(image, (IMAGE_H, IMAGE_W))
    input_image = input_image / 255.
    input_image = input_image[:,:,::-1]

    input_image = np.expand_dims(input_image, 0)

    netout = model.predict(input_image)
    boxes = decode_netout2(netout, 
                          obj_threshold=OBJ_THRESHOLD,
                          nms_threshold=NMS_THRESHOLD,
                          anchors=ANCHORS, 
                          nb_class=CLASS)    
    for box in boxes:
        x1, y1, x2, y2,new_probs  = box.xmin, box.ymin, box.xmax, box.ymax, box.c
        
        key = LABELS[box.get_label()]
        x1, y1, x2, y2 = int(round(x1*w)), int(round(y1*h)), int(round(x2*w)), int(round(y2*h))        

        if x1 < 10 or y1 < 10 or x2> w-10 or y2 > h - 10:
            continue  
        coord = str(x1)+','+str(y1)+' '+str(x2)+','+str(y1)+' '+ str(x1)+','+str(y2)+' '+ str(x2)+','+str(y2)
        
        nodeManager = write2xml( key, str(new_probs), coord)
        singleimage.appendChild(nodeManager)
        
    root.appendChild(singleimage)

doc.appendChild(root)
fp = open('Manager.xml', 'w')
doc.writexml(fp, indent='\t', addindent='\t', newl='\n', encoding="utf-8")

In [21]:
fp.close()

In [19]:
filename = 'utils.py'
exec(compile(open(filename, "rb").read(), filename, 'exec'))

In [31]:
-1//8

-1

In [ ]:
65.74